In [25]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout,InputLayer, BatchNormalization, GlobalAveragePooling2D 
from tensorflow.keras.layers import Input, Dense, LSTM, Conv3D, Flatten
from keras import models, layers
from tensorflow.keras.layers import LSTM, Dense , Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [26]:
SEQUENCE_LENGTH = 29
FEATURES = 51

In [27]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

def read_data_from_folder(folder_path, sequence_length=29):
    class_labels = os.listdir(folder_path)
    sequences = []
    labels = []
    for label_num_temp, person_folder in enumerate(class_labels):
        print(f"Running for person {person_folder}")
        person_folder_path = os.path.join(folder_path, person_folder)
        if not os.path.isdir(person_folder_path):
            continue
        
        for csv_file in os.listdir(person_folder_path):
            if not csv_file.endswith('.csv'):
                continue
            
            csv_file_path = os.path.join(person_folder_path, csv_file)
            df = pd.read_csv(csv_file_path)
            df = df.iloc[:, 2:]  # Select columns from the third column to the last
            num_frames = min(sequence_length, len(df))  # Ensure not more than sequence_length frames are taken
            
            sequence = df.iloc[:num_frames].values

            sequences.append(sequence)
            labels.append(label_num_temp) 

    return sequences, labels, class_labels


train_folder_path = './data/points/'
SEQUENCE_LENGTH = 30  # Define your desired sequence length

# Read data from the train folder
sequences, labels, class_labels = read_data_from_folder(train_folder_path, sequence_length=SEQUENCE_LENGTH)

Running for person ali
Running for person huzaifa
Running for person junaid
Running for person salman


In [28]:
print(class_labels)
print(labels)
print(np.array(sequences).shape)
# Assuming you have the labels stored in the 'labels' variable
# Compute the number of unique labels (num_classes)
num_classes = len(set(labels))
print(f"The total number of classes is: {num_classes}")

['ali', 'huzaifa', 'junaid', 'salman']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [29]:
sequences = np.array(sequences)
labels = np.array(labels)

In [30]:
X_Lstm, X_val_lstm, y_lstm, y_val_lstm = train_test_split(sequences, labels, test_size=0.25, random_state=42)
print(X_Lstm.shape)
print(y_lstm.shape)
print(X_val_lstm.shape)
print(y_val_lstm.shape)
print("The total number of labels is: ", len(np.unique(labels)))

(474, 29, 51)
(474,)
(159, 29, 51)
(159,)
The total number of labels is:  4


In [31]:
def simple_lstm(sequence_length, features, num_classes):
    model = tf.keras.Sequential()
    model.add(LSTM(64, input_shape=(sequence_length, features)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [32]:
lstm_input_shape = (29, 51)
num_classes = 4
lstm_model=simple_lstm(29,51,num_classes)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                29696     
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 29,956
Trainable params: 29,956
Non-trainable params: 0
_________________________________________________________________


In [33]:
firtting = lstm_model.fit(X_Lstm, y_lstm,
                          validation_data=(X_val_lstm, y_val_lstm),
                          epochs=100, verbose=1)


Epoch 1/100
15/15 [==============================] - 6s 39ms/step - loss: 1.2407 - accuracy: 0.5105 - val_loss: 1.1468 - val_accuracy: 0.5472
Epoch 2/100
15/15 [==============================] - 0s 13ms/step - loss: 1.0541 - accuracy: 0.6371 - val_loss: 1.1257 - val_accuracy: 0.5472
Epoch 3/100
15/15 [==============================] - 0s 11ms/step - loss: 1.0121 - accuracy: 0.6772 - val_loss: 1.0645 - val_accuracy: 0.6289
Epoch 4/100
15/15 [==============================] - 0s 11ms/step - loss: 0.9612 - accuracy: 0.6561 - val_loss: 1.0348 - val_accuracy: 0.5535
Epoch 5/100
15/15 [==============================] - 0s 9ms/step - loss: 0.9174 - accuracy: 0.7046 - val_loss: 0.9890 - val_accuracy: 0.5472
Epoch 6/100
15/15 [==============================] - 0s 9ms/step - loss: 0.8988 - accuracy: 0.6540 - val_loss: 0.9504 - val_accuracy: 0.5975
Epoch 7/100
15/15 [==============================] - 0s 9ms/step - loss: 0.8539 - accuracy: 0.7131 - val_loss: 0.9216 - val_accuracy: 0.5535
Epoch 8/1